In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\Users\arnab\miniconda3\envs\finetuning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
dataset=load_dataset("json", data_files="../processed-data/python_instruct.jsonl", split="train")

model_path="../models/base_model"

In [ ]:
model, tokenizer=FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
print(tokenizer)

In [ ]:
model=FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

In [ ]:
training_args=TrainingArguments(
    output_dir="../models/finetuned-python",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    warmup_steps=20,
    logging_steps=10,
    fp16=True,
    bf16=False,
    optim="paged_adamw_8bit",
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="../logs",
    report_to="none",
    lr_scheduler_type="cosine"
)

In [ ]:
def formatting_func(example):
    return example["prompt"]+example["response"]

In [ ]:
def format_dataset(example):
    return {"text": example["prompt"] + example["response"]}

dataset = dataset.map(format_dataset)

In [ ]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args
)

In [ ]:
trainer.train()

In [ ]:
model_path_python_trained="../models/finetuned-python"
resume_checkpoint="../models/finetuned-python/checkpoint-9834"

In [3]:
dataset = load_dataset("json", data_files="../processed-data/ds_coder_instruct.jsonl", split="train")

In [ ]:
model, tokenizer=FastLanguageModel.from_pretrained(
    model_name=resume_checkpoint,
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)